# WHATSAPP BLAST

#### CARA PAKAI

**Jalankan semua blok** di dalam **SETUP** saat pertama kali membuka program dengan cara menekan tombol **Shift + Enter** untuk setiap blok kode.  
  
Blok SETUP kedua akan membuka window browser baru yang berisi Whatsapp Web.  
setelah Whatsapp Web terbuka, lakukan login melalui QR di aplikasi WA Handphone.  
  
sebelum memulai program sesuaikan dulu isi dari file `data.csv` dengan format:
- kontak: nama kontak siswa yang tersimpan. **penulisan harus persis**
- nama: nama panggilan yang ingin digunakan.
- tunggakan: jumlah tunggakan.  
**jangan gunakan spasi antar tipe data, pemisah data hanya tanda koma (,)**  
**pisahkan tiap entri dengan menggunakan enter**

**contoh format data**  
kontak,namaPanggilan,tunggakan  
ichsanGO,ichsann,80  
nadiaGO,nayyy,120  

lakukan test menggunakan `dummy.csv`. isi dengan nomor dan nama sendiri

#### SETUP

In [2]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import ElementClickInterceptedException

import csv

In [3]:
# Start Web Driver
edge_options = Options()
service = Service(EdgeChromiumDriverManager().install())
driver = webdriver.Edge(service=service, options=edge_options)
driver.get("https://web.whatsapp.com")

In [4]:
def template(nama, tunggakan):
    pass

##############################################################
data = []


def load_data(data_file):
    global data
    data = []

    with open(data_file, mode="r", newline="") as file:
        csv_reader = csv.reader(file)
        next(csv_reader, None)
        for row in csv_reader:
            data.append(row)

def attempt_with_retries(func, retries=3, *args, **kwargs):
    for attempt in range(retries):
        try:
            return func(*args, **kwargs)
        except TimeoutException:
            print(f"Timeout: Attempt {attempt + 1} failed. Retrying...")
            if attempt == retries - 1:
                print("Max retry attempts reached. Exiting...")
                return None

##############################################################
def chatbot(data, action):
    # Click the search box and enter the contact name
    search_box = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//button[@class="_ai0b _ai08"]'))
    )
    search_box.click()

    # Enter the contact name into the search input
    span_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(
            (By.XPATH, '//p[@class="selectable-text copyable-text x15bjb6t x1n2onr6"]')
        )
    )
    span_element.send_keys(data)

    retry_attempts = 3  # Number of retry attempts
    for attempt in range(retry_attempts):
        try:
            # Wait for the contact to be clickable and click it
            locator_contact = f'//span[@title="{data}"]'
            contact_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, locator_contact))
            )
            
            # Scroll the contact into view
            driver.execute_script("arguments[0].scrollIntoView(true);", contact_button)
            
            # Ensure the element is not blocked by checking visibility and size
            if contact_button.is_displayed():
                contact_button.click()
                break  # If click is successful, exit the loop

        except ElementClickInterceptedException:
            print(f"Element click intercepted. Retrying... Attempt {attempt + 1}")
            if attempt == retry_attempts - 1:
                raise  # Re-raise exception if retry attempts fail
        except StaleElementReferenceException:
            print(f"Stale element reference exception. Retrying... Attempt {attempt + 1}")
            if attempt == retry_attempts - 1:
                raise  # Re-raise exception if retry attempts fail

    # Wait for the chat to open by checking the presence of the contact's name in the chat header
    attempt_with_retries(
        lambda: WebDriverWait(driver, 20).until(
            EC.text_to_be_present_in_element(
                (By.XPATH, '//div[@class="_amie"]/div/div/div/span'), data
            )
        )
    )

    # Find the message input box and click it
    message_box = attempt_with_retries(
        lambda: WebDriverWait(driver, 20).until(
            EC.presence_of_element_located(
                (By.XPATH, '//div[@aria-placeholder="Type a message"]/p')
            )
        )
    )
    message_box.click()

    # Perform the sender action (assuming 'sender' is an ActionChains object)
    action.perform()


##############################################################
actions = ActionChains(driver) # first time initiate


def teks(text):
    actions.send_keys(text)


def enter():
    actions.key_down(Keys.SHIFT).send_keys(Keys.ENTER).key_up(Keys.SHIFT)


def mulai():
    global actions
    actions = ActionChains(driver)
    

#############################################################
def kirim():
    print("Terkirim:")
    for row in data:
        kon, nama, tung, kemarin = row

        actions = ActionChains(driver)
        actions.send_keys(Keys.RETURN)
        chatbot(kon, actions)
        print("  ", nama)


def draft():
    print("Draft: ")
    for row in data:
        kon, nama, tung, kemarin = row

        mulai()
        template(nama, tung, kemarin)
        chatbot(kon, actions)
        print("  ", nama)


#### TEMPLATE

##### FUNGSI
- `teks(isi)` - untuk menambahkan teks ke dalam template. isi diganti dengan teks yang akan ditulis. teks diawali dan diakhiri dengan `"""`  
**cont.** `teks("""halo fahmi""")`

- `enter()` - buat nambah enter ke template

##### PENULISAN
- indentasi (menjorok ke dalam) harus sejajar semua  
**cont.**  
<code>def template(nama, tunggakan):</code>  
<code>&nbsp;&nbsp;teks("Reminder~")</code>  
<code>&nbsp;&nbsp;enter()</code>  
<code>&nbsp;&nbsp;enter()</code> 
   
`def template` **jangan diubah** yang diganti-ganti hanya baris kedua ke bawah

- penulisan nama dan tunggakan menggunakan `{}`  
**cont.**  
`teks("""Halo, {nama}. hari ini tunggakan kamu {tunggakan} yaa""")`


In [5]:
def template(nama, tunggakan, kemarin):

    kemarin_angka = int(kemarin)


    # teks("Reminder~")
    # enter()
    # enter()
    # if tunggakan == '0':
    #     teks(f"""{nama}, selamat yaa tunggakannya udah abis, tetep dikerjain ya soalnya biar tunggakannya bisa *0* terus""")
    # else:
    #     teks(f"""{nama},  tunggakannya masih ada yaa, cek di *GO Expert* tunggakannya sisa berapa. jangan lupa soalnya dikerjain biar targetnya bisa kekejar""")
        
    teks(f"""{nama},  TOBK jangan lupa dikerjain yaa, mayan buat nambah" pengerjaan soal juga""")
    # if kemarin_angka == 0 and tunggakan == '0':
    #     teks(f"""{nama}, kemaren belom ngerjain soal yaa. Jangan lupa buat dikerjain terus soalnya, biar tunggakannya tetep *0*""")
    # else:
    #     if kemarin_angka == 0:
    #         teks(f"""{nama}, kemaren belom ngerjain soal yaa. Jangan lupa buat dikerjain terus soalnya, biar tunggakannya ngga nambah teruss""")
    #     elif kemarin_angka < 40:
    #         teks(f"""{nama}, kemaren udah ngerjain *{kemarin}* soal yaa. tingkatin teruss! jangan mau ketinggalan sama yg lain!""")
    #     elif kemarin_angka == 69:
    #         teks(f"""{nama}, kemaren udah ngerjain *{kemarin}* soal yaa. noice~ pertahanin terus pengerjaanyaa""")
    #         enter()
    #         teks(f"""_ . .... .""")
    #     elif kemarin_angka < 300:
    #         teks(f"""{nama}, kemaren udah ngerjain *{kemarin}* soal yaa. well done! pertahanin terus pengerjaanyaa""")
    #     else:
    #         teks(f"""{nama}, kemaren udah ngerjain *{kemarin}* soal yaa. good job! nanti boleh ke bapa, tagih minta hadiah wkwk""")
    #     enter()
    #     if tunggakan == '0':
    #         teks(f"""buat sampe hari ini, tunggakannya udah habis. Selamatt!! pertahanin terus biar tetep di atas target '-')b""")
    #     else:
    #         teks(f"""Nah, sampe hari ini, tunggakannya ada *{tunggakan}* yaa. Beresin targetnya biar ngga dikejar" terus wkwk""")
    enter()
    enter()
    teks("Semangaatt '-')9")

#### PROGRAM


##### FUNGSI
- `load_data('data_file')` - untuk mengambil data load_data dari file csv. ganti data_file dengan `data.csv` atau `dummy.csv` sesuai kebutuhan dengan dibungkus koma di atas (') 
- `draft()` - untuk membuat draft pada chatroom
- `kirim()` - untuk mengirim chat  

**cont.**  
Hanya membuat draft  
```
load_data('dummy.csv')
draft()
```

Mengirim chat yang sudah menjadi draft
```
load_data('dummy.csv')  
kirim()
```

Langsung mengirimkan tanpa draft  
```
load_data('dummy.csv')
draft()
kirim()
```

In [6]:
load_data('data.csv')
draft()

Draft: 
   Keynina
   Hanif
   Abdee


NameError: name 'TimeoutException' is not defined